# 本库的基本用法

## 目前主要包含以下几个功能

- 计算原子温度
- 计算原子结构熵
- 计算中心对称参数
- 建立简单晶体模型
- 建立原子近邻列表

## 导入必要的package

In [1]:
import mdapy as mp # 无特殊说明本库简写为 mp
import numpy as np
import taichi as ti
ti.init(ti.cpu)

[Taichi] version 1.1.2, llvm 10.0.0, commit f25cf4a2, win, python 3.7.13
[Taichi] Starting on arch=x64


## 读取一个系统
[可在此处下载example.dump文件](http://yunpan.mushroomfire.com/s/GzTv)

In [2]:
system = mp.System('example.dump') 

## 查看系统数据

In [3]:
system.data

,id,type,x,y,z,vx,vy,vz,c_1,c_2
0,192295,3,-27.9096,42.0121,-22.2491,-2.35674,0.481940,-0.257686,5.06650,-3.71151
1,255932,5,-30.0338,43.2841,-22.1938,1.55510,-1.125920,-1.983760,5.16323,-3.69935
2,319374,2,-29.4179,42.0487,-20.1799,-1.56154,-0.105773,-1.052900,6.02419,-3.51732
3,383068,3,-27.3053,43.2945,-20.1681,1.75865,0.962938,-2.273980,5.15436,-3.90756
4,700644,2,-27.9671,44.5056,-22.1799,2.11749,-0.684720,-0.795077,6.15314,-3.85788
...,...,...,...,...,...,...,...,...,...,...
2287143,1715363,1,329.5120,257.9240,323.0480,1.93039,1.215740,-1.869160,5.89311,-3.92515
2287144,1905765,5,327.2950,256.6070,318.9410,2.16960,-4.590730,1.395140,4.94041,-4.08289
2287145,1966370,3,329.5330,257.9350,318.9310,2.65692,-3.539740,-1.125020,5.70667,-3.83374
2287146,2030121,1,328.9320,256.6390,321.0620,1.25297,0.868006,1.104770,5.42057,-3.39951


## 初始化邻域列表

In [4]:
neighbor = mp.Neighbor(system.pos, system.box, 5., max_neigh=60)

## 生成邻域列表

In [5]:
%%time
neighbor.compute()

Wall time: 3.18 s


## 初始化中心对称参数

In [6]:
CSP = mp.CentroSymmetryParameter(neighbor.pos_ti, neighbor.box_ti, neighbor.boundary, neighbor.verlet_list_ti, neighbor.distance_list_ti, 12)

## 计算中心对称参数

In [7]:
%%time
CSP.compute()

Wall time: 2.25 s


## 初始化原子构型熵

In [8]:
Entropy = mp.AtomicEntropy(neighbor.box_ti, neighbor.distance_list_ti, 5.)

## 计算原子构型熵

In [9]:
%%time
Entropy.compute()

Wall time: 6.73 s


## 初始化原子温度

In [10]:
Temperature = mp.AtomicTemperature(amass=np.array([58.933, 58.693, 55.847, 26.982, 63.546]), vel=system.vel, verlet_list = neighbor.verlet_list_ti, atype_list=system.data['type'].values)

## 计算原子温度

In [11]:
%%time
Temperature.compute()

Wall time: 336 ms


## 查看系统平均温度

In [12]:
Temperature.T.to_numpy().mean() # K

293.7486024263631

## 添加计算结果到系统中

In [13]:
system.data['csp'] = CSP.csp.to_numpy()
system.head[-1] = system.head[-1].strip() + ' csp'
system.data['entropy'] = Entropy.entropy.to_numpy()
system.head[-1] = system.head[-1].strip() + ' entropy'
system.data['temperature'] = Temperature.T.to_numpy()
system.head[-1] = system.head[-1].strip() + ' temperature'

## 保存计算结果到dump文件中

In [14]:
%%time
mp.write_dump(system)

Wall time: 30.3 s
